# Make a full overview of number counts by band by field

The reviewer requested that we produce number counts for each band on each field

In [11]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib as mpl
mpl.use('pdf')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
#plt.rc('figure', figsize=(10, 6))
from matplotlib_venn import venn3

import herschelhelp 
from herschelhelp.utils import clean_table

from astropy.table import Table

import pyvo as vo

import time

import yaml



import warnings
warnings.filterwarnings('ignore')

In [2]:
#Then we establish the VO connection to our database
service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap", 
                            maxrec=100000000,
                            hardlimit=100000000
                           )

In [3]:
fields = yaml.load(open('../../../dmu2/meta_main.yml', 'r'))['fields']

In [30]:
bands = [
    'mmt_g', 
    'omegacam_g', 
    'suprime_g', 
    'megacam_g', 
    'wfc_g', 
    'gpc1_g', 
    'decam_g', 
    '90prime_g', 
    'sdss_g',
    'isaac_k', 
    'moircs_k', 
    'ukidss_k', 
    'newfirm_k', 
    'wircs_k', 
    'hawki_k',
    'wircam_ks', 
    'vista_ks', 
    'moircs_ks', 
    'omega2000_ks', 
    'tifkam_ks'
]
mag_tables = {}


for band in bands:
    mag_tables[band] = {}
    for f in fields:
        field_name = f['name'].replace('NGP', 'HATLAS-NGP').replace('SGP', 'HATLAS-SGP')
        mag_tables[band].update({field_name: None})

In [5]:
start = time.time()
for band in bands:
    print(band)
    
    for f in fields:
        try:
            bands[band][] = Table.read('./data/{}.fits'.format(band))
            print("loaded {} from file ({} objects)".format(band, len(bands[band][0])))
            continue
        except FileNotFoundError"
            print("Querying VOX for {} mags".format(band))
    
        query = """
        SELECT 
        m_{}
        FROM herschelhelp.main
        WHERE herschelhelp.main.m_{} IS NOT NULL
        AND herschelhelp.main.field='{}'""".format(band, band, field)

        job = service.submit_job(query, maxrec=100000000)
        job.run()
        job_url = job.url
        job_result = vo.dal.tap.AsyncTAPJob(job_url)
        start_time = time.time()
        wait = 10.
        while job.phase == 'EXECUTING':
            #print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
            time.sleep(wait) 
            #wait *=2

        print(job.phase)
        result = job_result.fetch_result()
        bands[band].update({ f['name']: result.table})
        print("Band {} field {} done in {} seconds with {} objects".format(
            band, 
            f['name']                                                         
            round(time.time() - start_time), 
            len(bands[band][0])
        ))
    
print("Total time: {} seconds".format(round(time.time() - start)))

mmt_g
Job still running after 0 seconds.
Job still running after 11 seconds.
Job still running after 31 seconds.
Job still running after 71 seconds.
Job still running after 152 seconds.
Job still running after 312 seconds.
COMPLETED
Band mmt_g done in 637 seconds
omegacam_g
Job still running after 0 seconds.
Job still running after 11 seconds.
Job still running after 31 seconds.
Job still running after 71 seconds.
Job still running after 152 seconds.
Job still running after 312 seconds.
COMPLETED
Band omegacam_g done in 1626 seconds
suprime_g
Job still running after 0 seconds.
Job still running after 11 seconds.
Job still running after 31 seconds.
Job still running after 71 seconds.
Job still running after 152 seconds.
Job still running after 312 seconds.
COMPLETED
Band suprime_g done in 2627 seconds
megacam_g
Job still running after 0 seconds.
Job still running after 11 seconds.
Job still running after 31 seconds.
Job still running after 71 seconds.
Job still running after 152 seconds

DALServiceError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [9]:
write = True
read = False
for band in bands:
    if write:
        clean_table(bands[band][0]).write('./data/{}.fits'.format(band), overwrite = True)
        print('{} cleaned and written to ./data'.format(band))
    elif read:
        bands[band][0] = Table.read('./data/{}.fits'.format(band))
    

Converting column field type from object to string
Converting column field type from object to string
Converting column field type from object to string
Converting column field type from object to string
Converting column field type from object to string
Converting column field type from object to string
Converting column field type from object to string


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
depth_query = """
SELECT 
DISTINCT
hp_idx_o_10,"""
for band in bands:
    depth_query += " ferr_ap_{}_mean,".format(band)

depth_query = depth_queary.strip(',')
depth_query +="""FROM depth.main"""

job = service.submit_job(depth_query, maxrec=100000000)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
wait = 10.
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(wait) 
    wait *=2

print(job.phase)
result = job_result.fetch_result()
depth_result = result.table
   

## Plot the histograms

In [ ]:
h = np.histogram(mags, bins = 100)
bin_width = (np.abs(h[1][5] - h[1][4]) )
vals = plt.fill_between( h[1][:-1], h[0]/bin_width, alpha=0.5)
plt.xlim(20.,30.)
plt.ylim(0.,40000.)

In [ ]:
len(bands['mmt_g'][0])

In [ ]:
np.unique(bands['mmt_g'][0]['field'])

In [ ]:
len(bands['omegacam_g'][0])

In [ ]:
np.unique(bands['omegacam_g'][0]['field'])

In [ ]:
fig, ax = plt.subplots()


field = 'EGS'
area = 5
for band in ['mmt_g', 'omegacam_g']:
    mask = np.isfinite(bands[band][0]['m_'+band])
    mags = bands[band][0][mask]['m_'+band]
    if not np.sum(mask)==0:
        #vz.hist(table[name][mask], bins='scott', label=label, alpha=.5)
        h = np.histogram(mags, bins = 100)
        bin_width = (np.abs(h[1][5] - h[1][4]) )
        ax.fill_between( h[1][:-1], h[0]/bin_width, alpha=0.4)


ax.legend(loc=1, fontsize=8)

plt.xlim(20.,27.)
plt.xlabel("Magnitude [mag]")


#plt.ylim(0.,0.4)
plt.ylabel('Number [dex$^{-1}$]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)


#plt.savefig('./figs/numbers_g_en1.pdf', bbox_inches='tight')
#plt.savefig('./figs/numbers_g_en1.png', bbox_inches='tight')

In [ ]:
dim = [4,6]
fig, axes = plt.subplots(dim[1], dim[0], sharex=True, sharey=True)
plt.rcParams.update({'font.size': 12})


for n, f in enumerate(fields):
    f = f['name']
        
    x, y = np.floor_divide(n, dim[0]), np.remainder(n, dim[0])
    
    f_moc = MOC(filenam='../../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(f))
    
    
    for band in ['mmt_g', 'omegacam_g']:
        mask = np.isfinite(bands[band][0]['m_'+band])
        #mask &= (bands[band][0]['field'] == f)
        band_moc = MOC(
            depth_result[~np.isnan(depth_result['ferr_{}_mean'.format(band)])]['hp_idx_o_10']
        )
        area = band_moc.intersection( f_moc.flatten(order=10))
        mags = bands[band][0][mask]['m_'+band]
        if not np.sum(mask) == 0:
            h = np.histogram(mags, bins = 100)/area
            bin_width = (np.abs(h[1][5] - h[1][4]) )
            axes[x,y].fill_between( h[1][:-1], h[0]/bin_width, alpha=0.4, label=f)


    axes[x,y].legend(loc=1, fontsize=8)
   

    def set_axis_style(ax):
        axes[x,y].get_xaxis().set_tick_params(direction='out')
        axes[x,y].xaxis.set_ticks_position('bottom')
        axes[x,y].tick_params(axis='x', labelsize=8)
        #axes[x,y].set_xticks(np.arange(1, len(labels) + 1))
        #axes[x,y].set_xticklabels(labels)
        axes[x,y].set_xlim(20, 30)


    #axes[x,y].set_ylabel('log10( 5$\sigma$ Depths  [Jy] )')
    set_axis_style(axes[x,y])
    axes[x,y].set_ylim(0, 6000000)
    axes[x,y]..yscale('log')





    
axes[dim[1]-1,dim[0]-1].tick_params(axis='x', labelsize=8)
#axes[dim[1]-1,dim[0]-1].set_xlabel('band')
    
fig.text(0.5, 0.07, '$g$ magnitude [mag]', ha='center')
fig.text(0.04, 0.5, 'Number count [deg.$^2$ dex$^{-1}$]', va='center', rotation='vertical')

fig.set_size_inches(10, 12)
fig.subplots_adjust(hspace=0, wspace=0)

plt.rc('axes', labelsize=12)
plt.savefig('./figs/numbers_g_allfields.pdf', bbox_inches='tight')
plt.savefig('./figs/numbers_g_allfields.png', bbox_inches='tight')